In [1]:
# !pip install evaluate
# 参考1：https://github.com/EleutherAI/lm-evaluation-harness?tab=readme-ov-file#overview
# 参考2：https://github.com/meta-llama/llama-recipes/tree/main/tools/benchmarks/llm_eval_harness
# 参考3：https://github.com/meta-llama/llama-recipes/tree/main/tools/benchmarks/llm_eval_harness/meta_eval
# 参考4：https://huggingface.co/datasets/meta-llama/Llama-3.2-1B-evals

In [10]:
import torch
import torchvision

print("Torch version:", torch.__version__)
print("Torchvision version:", torchvision.__version__)
print(torch.version.cuda)

Torch version: 2.5.1
Torchvision version: 0.20.1
12.1


In [9]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('ggplot')

from trl import SFTTrainer
from transformers import TrainingArguments, TextStreamer
from unsloth.chat_templates import get_chat_template
from unsloth import FastLanguageModel
from datasets import Dataset
from unsloth import is_bfloat16_supported

# Saving model
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Warnings
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [4]:
# Use the following command to upgrade PyTorch, ensuring compatibility with CUDA 12.1
!conda install pytorch=2.5.1 torchvision torchaudio pytorch-cuda=12.1 -c pytorch -c nvidia


Retrieving notices: ...working... done
Channels:
 - pytorch
 - nvidia
 - defaults
Platform: linux-64
Solving environment: done

## Package Plan ##

  environment location: /home/shaoxuan/anaconda3

  added / updated specs:
    - pytorch-cuda=12.1
    - pytorch=2.5.1
    - torchaudio
    - torchvision


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    cuda-cudart-12.1.105       |                0         189 KB  nvidia
    cuda-cupti-12.1.105        |                0        15.4 MB  nvidia
    cuda-libraries-12.1.0      |                0           2 KB  nvidia
    cuda-nvrtc-12.1.105        |                0        19.7 MB  nvidia
    cuda-nvtx-12.1.105         |                0          57 KB  nvidia
    cuda-opencl-12.6.77        |                0          25 KB  nvidia
    cuda-runtime-12.1.0        |                0           1 KB  nvidia
    libcublas-12.1.0.26        |         

# 可以忽略的内容

In [4]:
# 参考https://huggingface.co/datasets/meta-llama/Llama-3.2-1B-evals
from datasets import load_dataset
data = load_dataset("meta-llama/Llama-3.2-1B-evals",
        name="Llama-3.2-1B-evals__metrics",
        split="latest"
)

In [5]:
benchmark_list=data['benchmark_label']
# 找到所有benchmark_label="MMLU"的下标
indices = [index for index, value in enumerate(benchmark_list) if value == "MMLU"]

In [6]:
# 选取MMLU评测集中第一个评测任务
data[indices[0]]

{'benchmark_label': 'MMLU',
 'metric_tag': 'world_religions/nll_token_target_norm',
 'metric_value_computed': 1.3,
 'metric_type': 'subtask_metric',
 'eval_config': {'top_p': '0',
  'seed': '42',
  'max_gen_len': '0',
  'top_k': '0',
  'temperature': '0.0',
  'prompt_fn': 'functools.partial(<function jinja_format at 0x7f909c2f1480>, \'The following are multiple choice questions (with answers) about {{ subject }}.\\n\\n{% for x in few_shot -%}\\n{{ x["question"] }}\\nA. {{ x["choices"]["A"] }}\\nB. {{ x["choices"]["B"] }}\\nC. {{ x["choices"]["C"] }}\\nD. {{ x["choices"]["D"] }}\\nAnswer: {{ x["answer"] }}\\n\\n{% endfor -%}\\n{{ question }}\\nA. {{ choices["A"] }}\\nB. {{ choices["B"] }}\\nC. {{ choices["C"] }}\\nD. {{ choices["D"] }}\\nAnswer: {{ choice_text }}\', subject=\'world religions\')',
  'num_few_shot': '5',
  'max_prompt_len': '3840',
  'num_generations': None,
  'return_logprobs': None}}

In [7]:
prompt_template=','.join(data[indices[0]]['eval_config']['prompt_fn'].split(',')[1:-1])
prompt_template

' \'The following are multiple choice questions (with answers) about {{ subject }}.\\n\\n{% for x in few_shot -%}\\n{{ x["question"] }}\\nA. {{ x["choices"]["A"] }}\\nB. {{ x["choices"]["B"] }}\\nC. {{ x["choices"]["C"] }}\\nD. {{ x["choices"]["D"] }}\\nAnswer: {{ x["answer"] }}\\n\\n{% endfor -%}\\n{{ question }}\\nA. {{ choices["A"] }}\\nB. {{ choices["B"] }}\\nC. {{ choices["C"] }}\\nD. {{ choices["D"] }}\\nAnswer: {{ choice_text }}\''

In [8]:
# llama3.2在mmlu数据集上用的提示词模板
print(prompt_template.replace("\\n", "\n"))

 'The following are multiple choice questions (with answers) about {{ subject }}.

{% for x in few_shot -%}
{{ x["question"] }}
A. {{ x["choices"]["A"] }}
B. {{ x["choices"]["B"] }}
C. {{ x["choices"]["C"] }}
D. {{ x["choices"]["D"] }}
Answer: {{ x["answer"] }}

{% endfor -%}
{{ question }}
A. {{ choices["A"] }}
B. {{ choices["B"] }}
C. {{ choices["C"] }}
D. {{ choices["D"] }}
Answer: {{ choice_text }}'


# 通过lm-evaluation-harness评估LLM

参考https://github.com/meta-llama/llama-recipes/tree/main/tools/benchmarks/llm_eval_harness/meta_eval \
在tools\benchmarks\llm_eval_harness\meta_eval\eval_config.yaml中发现最新的公开config只支持llama3.1，无法复现llama3.2。而llama3.1中最小的是8B模型，该LLM太大，所以没法直接利用上面网址中的方法复现llama3.1。\
我们只能通过lm-evaluation-harness评估LLM，因为缺少config(比如提示词模板)和评估时的各种trick，所以难以复现结果 \
下面在mmlu_abstract_algebra上评估llama-3.2-1B和我们之前上传的微调模型，发现微调后效果变差，符合预期

In [9]:
# 参考：https://github.com/EleutherAI/lm-evaluation-harness/blob/main/docs/interface.md
!lm_eval --tasks list

2024-11-13 00:56:30.300262: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-13 00:56:30.300304: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-13 00:56:30.300863: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-13 00:56:30.304657: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-13 00:56:30.724646: W tensorflow/compiler/tf2

In [10]:
!lm_eval --model hf \
    --model_args "pretrained=meta-llama/Llama-3.2-1B" \
    --tasks mmlu_abstract_algebra\
    --device cuda:0 \
    --batch_size 8

2024-11-13 00:56:38.085410: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-13 00:56:38.085457: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-13 00:56:38.086061: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-13 00:56:38.089890: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-13 00:56:38.513371: W tensorflow/compiler/tf2

In [11]:
# 执行这一步需要先把微调后的模型push_to_hub
!lm_eval --model hf \
    --model_args "pretrained=yuntaozh/1B_finetuned_llama3.2" \
    --tasks mmlu_abstract_algebra\
    --device cuda:0 \
    --batch_size 8

2024-11-13 00:56:51.662191: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-13 00:56:51.662232: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-13 00:56:51.662810: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-13 00:56:51.666442: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-13 00:56:52.084472: W tensorflow/compiler/tf2

In [3]:
!lm_eval --model hf \
    --model_args "pretrained=meta-llama/Llama-3.2-1B" \
    --tasks mmlu_abstract_algebra\
    --device cuda:0 \
    --batch_size 8

2024-11-14:11:57:55,912 INFO     [__main__.py:279] Verbosity set to INFO
2024-11-14:11:57:58,170 INFO     [__init__.py:459] The tag 'arc_ca' is already registered as a group, this tag will not be registered. This may affect tasks you want to call.
2024-11-14:11:57:58,180 INFO     [__init__.py:459] The tag 'arc_ca' is already registered as a group, this tag will not be registered. This may affect tasks you want to call.
2024-11-14:11:57:58,796 INFO     [__main__.py:376] Selected Tasks: ['mmlu_abstract_algebra']
2024-11-14:11:57:58,797 INFO     [evaluator.py:164] Setting random seed to 0 | Setting numpy seed to 1234 | Setting torch manual seed to 1234 | Setting fewshot manual seed to 1234
2024-11-14:11:57:58,797 INFO     [evaluator.py:201] Initializing hf model, with arguments: {'pretrained': 'meta-llama/Llama-3.2-1B'}
2024-11-14:11:57:58,984 INFO     [huggingface.py:131] Using device 'cuda:0'
config.json: 100%|█████████████████████████████| 843/843 [00:00<00:00, 9.45MB/s]
2024-11-14:11:

In [7]:
!lm_eval --model hf \
    --model_args "pretrained=meta-llama/Llama-3.2-1B" \
    --tasks hellaswag \
    --device cuda:0 \
    --batch_size 8


2024-11-14:14:03:28,043 INFO     [__main__.py:279] Verbosity set to INFO
2024-11-14:14:03:30,866 INFO     [__init__.py:459] The tag 'arc_ca' is already registered as a group, this tag will not be registered. This may affect tasks you want to call.
2024-11-14:14:03:30,879 INFO     [__init__.py:459] The tag 'arc_ca' is already registered as a group, this tag will not be registered. This may affect tasks you want to call.
2024-11-14:14:03:31,687 INFO     [__main__.py:376] Selected Tasks: ['hellaswag']
2024-11-14:14:03:31,690 INFO     [evaluator.py:164] Setting random seed to 0 | Setting numpy seed to 1234 | Setting torch manual seed to 1234 | Setting fewshot manual seed to 1234
2024-11-14:14:03:31,690 INFO     [evaluator.py:201] Initializing hf model, with arguments: {'pretrained': 'meta-llama/Llama-3.2-1B'}
2024-11-14:14:03:31,963 INFO     [huggingface.py:131] Using device 'cuda:0'
2024-11-14:14:03:32,143 INFO     [huggingface.py:484] Using model type 'default'
2024-11-14:14:03:32,436 IN

In [8]:
!lm_eval --model hf \
    --model_args "pretrained=meta-llama/Llama-3.2-1B" \
    --tasks mmlu \
    --device cuda:0 \
    --batch_size 8

2024-11-14:14:07:29,072 INFO     [__main__.py:279] Verbosity set to INFO
2024-11-14:14:07:31,279 INFO     [__init__.py:459] The tag 'arc_ca' is already registered as a group, this tag will not be registered. This may affect tasks you want to call.
2024-11-14:14:07:31,288 INFO     [__init__.py:459] The tag 'arc_ca' is already registered as a group, this tag will not be registered. This may affect tasks you want to call.
2024-11-14:14:07:31,893 INFO     [__main__.py:376] Selected Tasks: ['mmlu']
2024-11-14:14:07:31,894 INFO     [evaluator.py:164] Setting random seed to 0 | Setting numpy seed to 1234 | Setting torch manual seed to 1234 | Setting fewshot manual seed to 1234
2024-11-14:14:07:31,894 INFO     [evaluator.py:201] Initializing hf model, with arguments: {'pretrained': 'meta-llama/Llama-3.2-1B'}
2024-11-14:14:07:32,087 INFO     [huggingface.py:131] Using device 'cuda:0'
2024-11-14:14:07:32,231 INFO     [huggingface.py:484] Using model type 'default'
2024-11-14:14:07:32,541 INFO   